In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

import sklearn as sk
from sklearn.model_selection import TimeSeriesSplit

In [193]:
# Download the data from Yahoo Finance (^SOX and ^VIX)
start_date='2011-12-23'
end_date='2023-01-11'

data = yf.download('^SOX ^VIX', start=start_date, end=end_date, interval='1d')['Adj Close']
data.index = pd.to_datetime(data.index)
data['^VIX^2'] = data['^VIX'].pow(2)

# Calculate the log returns
data['^SOX: Log_Returns'] = np.log(data['^SOX'].pct_change() + 1)
data['^VIX: Log_Returns'] = np.log(data['^VIX'].pct_change() + 1)
data['^VIX^2: Log_Returns'] = np.log(data['^VIX^2'].pct_change() + 1)

data['^SOX: Weekly_RV'] = np.sqrt((data['^SOX: Log_Returns']**2).rolling(5).sum())
data['^SOX: Next_Weekly_RV'] = np.sqrt((data['^SOX: Log_Returns']**2).rolling(5).sum()).shift(-6)
data = data.dropna()
data.describe()

[*********************100%%**********************]  2 of 2 completed


Ticker,^SOX,^VIX,^VIX^2,^SOX: Log_Returns,^VIX: Log_Returns,^VIX^2: Log_Returns,^SOX: Weekly_RV,^SOX: Next_Weekly_RV
count,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000
mean,1369.225059,17.854458,368.574568,0.000700,-0.000028,-0.000055,0.034379,0.034444
std,972.320327,7.057683,414.575254,0.018118,0.078635,0.157270,0.021453,0.021466
min,351.280029,9.140000,83.539606,-0.173119,-0.299831,-0.599662,0.003846,0.003846
25%,616.449982,13.220000,174.768407,-0.008255,-0.044182,-0.088363,0.020779,0.020800
50%,1083.289978,15.935000,253.924248,0.001454,-0.006686,-0.013373,0.029266,0.029367
75%,1845.120026,20.712499,429.007639,0.010203,0.035784,0.071568,0.042783,0.042919
max,4039.510010,82.690002,6837.636504,0.105753,0.768245,1.536490,0.271085,0.271085


In [194]:
data.head(10)

Ticker,^SOX,^VIX,^VIX^2,^SOX: Log_Returns,^VIX: Log_Returns,^VIX^2: Log_Returns,^SOX: Weekly_RV,^SOX: Next_Weekly_RV
Date,,,,,,,,
2012-01-03,368.421753,22.969999,527.620868,0.011350,-0.018547,-0.037094,0.021267,0.026753
2012-01-04,368.481720,22.219999,493.728369,0.000163,-0.033196,-0.066392,0.021207,0.024366
2012-01-05,373.989044,21.480000,461.390380,0.014835,-0.033871,-0.067741,0.021205,0.032283
2012-01-06,375.038544,20.629999,425.596865,0.002802,-0.040376,-0.080752,0.019325,0.026164
2012-01-09,382.404968,21.070000,443.944887,0.019451,0.021104,0.042208,0.027114,0.054788
2012-01-10,386.093170,20.690001,428.076122,0.009599,-0.018200,-0.036399,0.026428,0.057642
2012-01-11,387.702423,21.049999,443.102468,0.004159,0.017250,0.034500,0.026753,0.057041
2012-01-12,391.560547,20.469999,419.020872,0.009902,-0.027940,-0.055880,0.024366,0.052976
2012-01-13,383.284546,20.910000,437.228094,-0.021363,0.021267,0.042534,0.032283,0.052897


In [195]:
data.tail(10)

Ticker,^SOX,^VIX,^VIX^2,^SOX: Log_Returns,^VIX: Log_Returns,^VIX^2: Log_Returns,^SOX: Weekly_RV,^SOX: Next_Weekly_RV
Date,,,,,,,,
2022-12-16,2636.100098,22.620001,511.664438,-0.009555,-0.009241,-0.018482,0.053228,0.052457
2022-12-19,2599.860107,22.420000,502.656403,-0.013843,-0.008881,-0.017762,0.050765,0.054155
2022-12-20,2583.639893,21.480000,461.390380,-0.006258,-0.042831,-0.085662,0.048866,0.058801
2022-12-21,2644.500000,20.070000,402.804888,0.023283,-0.067896,-0.135792,0.051769,0.040179
2022-12-22,2533.330078,21.969999,482.680870,-0.042947,0.090452,0.180903,0.052045,0.042030
2022-12-23,2535.489990,20.870001,435.556935,0.000852,-0.051365,-0.102730,0.051167,0.046591
2022-12-27,2490.169922,21.650000,468.722483,-0.018036,0.036693,0.073385,0.052457,0.048492
2022-12-28,2453.489990,22.139999,490.179573,-0.014839,0.022380,0.044761,0.054155,0.058023
2022-12-29,2534.949951,21.440001,459.673623,0.032662,-0.032128,-0.064255,0.058801,0.061054


In [196]:
# rename the columns (delete special characters)
data = data.rename(columns=lambda x: x.replace('^', '').replace(':', ''))
data.columns

Index(['SOX', 'VIX', 'VIX2', 'SOX Log_Returns', 'VIX Log_Returns',
       'VIX2 Log_Returns', 'SOX Weekly_RV', 'SOX Next_Weekly_RV'],
      dtype='object', name='Ticker')

### Data Preprocessing
1. Wisnorization
2. Min Max Scaling

In [197]:
# transform numerical data with wisnorization
from scipy.stats.mstats import winsorize
data = pd.DataFrame(winsorize(np.array(data), limits=[0.05, 0.05]), columns=data.columns, index=data.index)
data.describe()

Ticker,SOX,VIX,VIX2,SOX Log_Returns,VIX Log_Returns,VIX2 Log_Returns,SOX Weekly_RV,SOX Next_Weekly_RV
count,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000
mean,969.024283,17.854458,346.973857,0.000761,0.004695,0.024098,0.034379,0.034444
std,380.951457,7.057683,258.872269,0.017738,0.071092,0.128284,0.021453,0.021466
min,351.280029,9.140000,83.539606,-0.077558,-0.077558,-0.077558,0.003846,0.003846
25%,616.449982,13.220000,174.768407,-0.008255,-0.044182,-0.077558,0.020779,0.020800
50%,1083.289978,15.935000,253.924248,0.001454,-0.006686,-0.013373,0.029266,0.029367
75%,1351.849976,20.712499,429.007639,0.010203,0.035784,0.071568,0.042783,0.042919
max,1351.849976,82.690002,1351.849976,0.105753,0.768245,1.536490,0.271085,0.271085


In [198]:
# transform numerical data with min-max scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
data.describe()

Ticker,SOX,VIX,VIX2,SOX Log_Returns,VIX Log_Returns,VIX2 Log_Returns,SOX Weekly_RV,SOX Next_Weekly_RV
count,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000
mean,0.617392,0.118483,0.207705,0.427249,0.097249,0.062982,0.114252,0.114497
std,0.380734,0.095958,0.204108,0.096763,0.084053,0.079480,0.080277,0.080325
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.265019,0.055472,0.071929,0.378064,0.039461,0.000000,0.063362,0.063441
50%,0.731593,0.092386,0.134340,0.431028,0.083792,0.039767,0.095121,0.095500
75%,1.000000,0.157342,0.272384,0.478755,0.134005,0.092393,0.145700,0.146210
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [280]:
# Create a pipeline to transform the data (winsozation and min-max scaling)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

def winsorize_data(data):
    return winsorize(np.array(data), limits=[0.05, 0.05])

def min_max_scale_data(data):
    return scaler.fit_transform(data)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', FunctionTransformer(winsorize_data), data.columns),
        ('num2', FunctionTransformer(min_max_scale_data), data.columns)
    ])

def data_preprocess_transform(data):
    data_tmp = preprocessor.fit_transform(data)
    return pd.DataFrame(data_tmp[:, :-1*len(data.columns)], columns=data.columns, index=data.index)

data = data_preprocess_transform(data)

### Training and Testing Dataset Preparation
1. Training Data : 2012-2020
2. Testing Data: 2021-2022

In [199]:
# Split the data into training and test sets (training data from 2012 to 2020, test data from 2021 to 2022)
data_train = data.loc['2012-01-01':'2020-12-31']
data_test = data.loc['2021-01-01':'2022-12-31']

### Base Model Attributes(X) and Target(Y)
1. X \
    1.1 Index Value: ^SOX, ^VIX \
    1.2 Square Value: ^VIX^2 \
    1.3 Log Return: ^SOX, ^VIX, ^VIX^2
    
2. y: Weekly Relative Volatility of ^SOX

In [200]:
# Split the data into features and target
X_base_train = data_train.drop(columns=['SOX Next_Weekly_RV'])
y_train = data_train[['SOX Next_Weekly_RV']]
X_base_test = data_test.drop(columns=['SOX Next_Weekly_RV'])
y_test = data_test[['SOX Next_Weekly_RV']]

### Benchmark Model Training and Performance
1. Decesion Tree Regressor
2. Linear Regression

performance matrix
1. Root Mean Square Error
2. R-square

In [201]:
# Create a decision tree regressor model and use parameters from the grid search
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

model = DecisionTreeRegressor(random_state=42)
model.fit(X_base_train, y_train)

param_grid = {
    'max_depth': range(1, 5),
    'min_samples_split': range(2, 5),
    'min_samples_leaf': range(3, 8),
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=-1)
grid_search.fit(X_base_train, y_train)

# Report the best model parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")

Best parameters: {'max_depth': 2, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best score: -0.018878200773558507


In [202]:
# Predict the target variable for the test set and calculate the RMSE and R2 score
from sklearn.metrics import mean_squared_error, r2_score

model = grid_search.best_estimator_
y_pred = model.predict(X_base_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 score: {r2}")

RMSE: 0.08272153325344428
R2 score: -0.23359850684181271


In [203]:
# Create a linear regression model
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_base_train, y_train)

LinearRegression()

In [204]:
# Predict the target variable for the test set and calculate the RMSE and R2 score
y_pred = model.predict(X_base_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 score: {r2}")

RMSE: 0.06468373300803408
R2 score: 0.2457299982283111


### LightGBM Model Training and Performance
Hyperparamter tuning: Optuna

Time series split = 5

Objective function: Mean(Root Mean Square Error)

Performance matrix:
1. Root Mean Square Error
2. R-square

Reference:\
https://forecastegy.com/posts/how-to-use-optuna-to-tune-lightgbm-hyperparameters/

In [416]:
model_result = pd.DataFrame(columns=['Name', 'Stydy', 'Model', 'RMSE', 'R2'])

In [401]:
tscv = TimeSeriesSplit(n_splits=5)

In [405]:
import lightgbm as lgb
import optuna

def objective(trial, X, y):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 200,
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**4),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 20),
        "gamma": trial.suggest_int("gamma", 5, 20),
    }

    rmse_scores = np.array([])
    model = lgb.LGBMRegressor(**params)

    for train_index, val_index in tscv.split(X):
        # print("TRAIN:", train_index, "TEST:", val_index)
        X_t, X_val = X.iloc[train_index], X.iloc[val_index]
        y_t, y_val = y.iloc[train_index], y.iloc[val_index]
        model.fit(X_t, y_t)
        predictions = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, predictions))
        rmse_scores = np.append(rmse_scores, rmse)
        # r2 = r2_score(y_val, predictions)
        # r2_scores = np.append(r2_scores, r2)
    return rmse_scores.mean()

In [417]:
def lightgbm_train_performance(model_name, bool_new_data, new_features_data):
    # Split the data into training and test sets (training data from 2012 to 2020, test data from 2021 to 2022)
    if bool_new_data:
        data_train = pd.concat([data, new_features_data], axis=1).loc['2012-01-01':'2020-12-31'].dropna()
        data_test = pd.concat([data, new_features_data], axis=1).loc['2021-01-01':'2022-12-31'].dropna()
    else:
        data_train = data.loc['2012-01-01':'2020-12-31'].dropna()
        data_test = data.loc['2021-01-01':'2022-12-31'].dropna()

    # Create X and y
    X_train = data_train.drop(columns=['SOX Next_Weekly_RV'])
    y_train = data_train[['SOX Next_Weekly_RV']]
    X_test = data_test.drop(columns=['SOX Next_Weekly_RV'])
    y_test = data_test[['SOX Next_Weekly_RV']]

    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=30)
    
    # Create a LightGBM model with the best parameters
    best_params = study.best_params
    model = lgb.LGBMRegressor(**best_params)
    model.fit(X_train, y_train)

    # Predict the target variable for the test set and calculate the RMSE and R2 score
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # print(f"RMSE: {rmse}")
    # print(f"R2 score: {r2}")

    model_result.loc[len(model_result)] = ['LightGBM_'+model_name, study, model, rmse, r2]    

In [ ]:
for i in range(10):
    lightgbm_train_performance('Baseline', False, None)

### Feature Engineering
1. log of lag weekly realized volatility
2. square of lag weekly realized volatility
3. log of VIX
4. log of past 5 days VIX
5. SOX High-Low Spread
6. SOX Open-Close Spread

In [369]:
data_lag = pd.DataFrame()
for i in range(1, 15):
    # Add new columns name 'SOX Weekly_RV Lag' (8 periods lag)
    data_lag[f'SOX Weekly_RV Lag {i}'] = data['SOX Weekly_RV'].shift(i)
    # Add new columns name 'SOX Weekly_Log_RV Lag' (8 periods lag)
    data_lag[f'SOX Weekly_Log_RV Lag {i}'] = np.log(data['SOX Weekly_RV']).shift(i)
    # Add new columns name 'SOX Weekly_Squared_RV Lag' (8 periods lag)
    data_lag[f'SOX Weekly_Squared_RV Lag {i}'] = (data['SOX Weekly_RV']**2).shift(i)

data_lag

c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr

,SOX Weekly_RV Lag 1,SOX Weekly_Log_RV Lag 1,SOX Weekly_Squared_RV Lag 1,SOX Weekly_RV Lag 2,SOX Weekly_Log_RV Lag 2,SOX Weekly_Squared_RV Lag 2,SOX Weekly_RV Lag 3,SOX Weekly_Log_RV Lag 3,SOX Weekly_Squared_RV Lag 3,SOX Weekly_RV Lag 4,...,SOX Weekly_Squared_RV Lag 11,SOX Weekly_RV Lag 12,SOX Weekly_Log_RV Lag 12,SOX Weekly_Squared_RV Lag 12,SOX Weekly_RV Lag 13,SOX Weekly_Log_RV Lag 13,SOX Weekly_Squared_RV Lag 13,SOX Weekly_RV Lag 14,SOX Weekly_Log_RV Lag 14,SOX Weekly_Squared_RV Lag 14
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,0.065188,-2.730480,0.004249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-05,0.064964,-2.733919,0.004220,0.065188,-2.730480,0.004249,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-06,0.064959,-2.734000,0.004220,0.064964,-2.733919,0.004220,0.065188,-2.730480,0.004249,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,0.057922,-2.848666,0.003355,0.064959,-2.734000,0.004220,0.064964,-2.733919,0.004220,0.065188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,0.180359,-1.712806,0.032529,0.179326,-1.718549,0.032158,0.168465,-1.781029,0.028380,0.175571,...,0.017723,0.099093,-2.311692,0.009820,0.226688,-1.484182,0.051387,0.209854,-1.561342,0.044039
2022-12-27,0.177075,-1.731182,0.031356,0.180359,-1.712806,0.032529,0.179326,-1.718549,0.032158,0.168465,...,0.016855,0.133129,-2.016435,0.017723,0.099093,-2.311692,0.009820,0.226688,-1.484182,0.051387
2022-12-28,0.181902,-1.704288,0.033088,0.177075,-1.731182,0.031356,0.180359,-1.712806,0.032529,0.179326,...,0.020689,0.129829,-2.041540,0.016855,0.133129,-2.016435,0.017723,0.099093,-2.311692,0.009820


In [ ]:
for i in range(10):
    lightgbm_train_performance('SOX RV Lag', True, data_lag)

In [376]:
# Create new dataframe to store new features for Lof of VIX and Log of past 5-day VIX
data_VIX = pd.DataFrame()
data_VIX['VIX Log'] = np.log(data['VIX'])
for i in range(1, 21, 5):
    data_VIX[f'VIX Log {i} Day Mean'] = np.log(data['VIX'].rolling(i).mean())
    data_VIX[f'VIX Log {i} Day Sum'] = np.log(data['VIX'].rolling(i).sum())

data_VIX

c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\LiaoYF\anaconda3\envs\FT5005MachineLearningEnv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,VIX Log,VIX Log 1 Day Mean,VIX Log 1 Day Sum,VIX Log 6 Day Mean,VIX Log 6 Day Sum,VIX Log 11 Day Mean,VIX Log 11 Day Sum,VIX Log 16 Day Mean,VIX Log 16 Day Sum
Date,,,,,,,,,
2012-01-03,-1.671125,-1.671125,-1.671125,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,-1.726881,-1.726881,-1.726881,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-05,-1.785120,-1.785120,-1.785120,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-06,-1.856488,-1.856488,-1.856488,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,-1.818909,-1.818909,-1.818909,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022-12-23,-1.835816,-1.835816,-1.835816,-1.777719,0.014041,-1.731339,0.666556,-1.750035,1.022554
2022-12-27,-1.771437,-1.771437,-1.771437,-1.790808,0.000951,-1.739611,0.658284,-1.737449,1.035140
2022-12-28,-1.733016,-1.733016,-1.733016,-1.794619,-0.002859,-1.759949,0.637946,-1.730759,1.041830


In [ ]:
for i in range(10):
    lightgbm_train_performance('VIX Log Value', True, data_VIX)

In [226]:
# Download the ^SOX data from Yahoo Finance (Open, High, Low, Close)
data_spread = yf.download('^SOX', start=start_date, end=end_date, interval='1d')[['Open', 'High', 'Low', 'Close']]
data_spread.index = pd.to_datetime(data_spread.index)

# Calculate the spread between the High and Low prices, open and close prices
data_spread['Spread'] = data_spread['High'] - data_spread['Low']
data_spread['Open_Close_Diff'] = data_spread['Open'] - data_spread['Close']

data_spread

[*********************100%%**********************]  1 of 1 completed
C:\Users\LiaoYF\AppData\Local\Temp\ipykernel_32200\3536122909.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_spread['Spread'] = data_spread['High'] - data_spread['Low']


,Open,High,Low,Close,Spread,Open_Close_Diff
Date,,,,,,
2011-12-23,367.920013,368.820007,364.290009,368.600006,4.529999,-0.679993
2011-12-27,367.480011,371.019989,366.470001,368.010010,4.549988,-0.529999
2011-12-28,367.820007,368.290009,362.079987,362.589996,6.210022,5.230011
2011-12-29,363.989990,366.489990,362.220001,365.929993,4.269989,-1.940002
2011-12-30,365.690002,367.220001,364.250000,364.440002,2.970001,1.250000
...,...,...,...,...,...,...
2023-01-04,2552.350098,2582.270020,2523.389893,2569.459961,58.880127,-17.109863
2023-01-05,2538.459961,2556.300049,2517.149902,2518.520020,39.150146,19.939941
2023-01-06,2553.399902,2649.379883,2519.179932,2636.100098,130.199951,-82.700195


In [365]:
# Add new columns of lag spread and lag open-close difference (8 periods lag)
data_spread_lag = pd.DataFrame()
for i in range(0, 17, 2):
    data_spread_lag[f'Spread Lag {i}'] = data_spread['Spread'].shift(i)
    data_spread_lag[f'Open_Close_Diff Lag {i}'] = data_spread['Open_Close_Diff'].shift(i)

data_spread_lag

,Spread Lag 0,Open_Close_Diff Lag 0,Spread Lag 2,Open_Close_Diff Lag 2,Spread Lag 4,Open_Close_Diff Lag 4,Spread Lag 6,Open_Close_Diff Lag 6,Spread Lag 8,Open_Close_Diff Lag 8,Spread Lag 10,Open_Close_Diff Lag 10,Spread Lag 12,Open_Close_Diff Lag 12,Spread Lag 14,Open_Close_Diff Lag 14,Spread Lag 16,Open_Close_Diff Lag 16
Date,,,,,,,,,,,,,,,,,,
2012-01-06,4.640015,-1.549988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,6.620026,-4.889984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-10,3.929993,0.809998,4.640015,-1.549988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-11,4.660004,-3.150024,6.620026,-4.889984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-12,6.179993,-2.720001,3.929993,0.809998,4.640015,-1.549988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-04,58.880127,-17.109863,54.959961,-39.380127,59.239990,21.149902,46.060059,-22.100098,51.949951,-42.709961,62.000000,34.649902,74.129883,67.280029,124.180176,77.469971,55.959961,14.070068
2023-01-05,39.150146,19.939941,85.399902,73.689941,58.209961,-41.439941,36.600098,25.650146,106.650146,55.770020,42.360107,-15.349854,57.670166,10.179932,98.199951,41.359863,80.000000,-69.109863
2023-01-06,130.199951,-82.700195,58.880127,-17.109863,54.959961,-39.380127,59.239990,21.149902,46.060059,-22.100098,51.949951,-42.709961,62.000000,34.649902,74.129883,67.280029,124.180176,77.469971


In [ ]:
for i in range(10):
    lightgbm_train_performance('SOX Spread', True, data_spread_lag)

In [ ]:
data_all_features = pd.concat([pd.concat([data_lag, data_VIX], axis=1), data_spread_lag], axis=1)

for i in range(10):
    lightgbm_train_performance('All', True, data_all_features)

### Model Performance Analysis

In [432]:
# Take the best model of each model name and print the result in descending order of R2 score
model_result.sort_values(by='R2', ascending=False).groupby('Name').first().sort_values(by='R2', ascending=False)

,Stydy,Model,RMSE,R2
Name,,,,
LightGBM_All,<optuna.study.study.Study object at 0x000002AA...,LGBMRegressor(colsample_bytree=0.8121778205506...,0.064781,0.243463
LightGBM_SOX RV Lag,<optuna.study.study.Study object at 0x000002AA...,LGBMRegressor(colsample_bytree=0.7945143191898...,0.071599,0.075829
LightGBM_Baseline,<optuna.study.study.Study object at 0x000002AA...,LGBMRegressor(colsample_bytree=0.5172360551584...,0.072637,0.048843
LightGBM_SOX Spread,<optuna.study.study.Study object at 0x000002AA...,LGBMRegressor(colsample_bytree=0.6698388023356...,0.073875,0.016133
LightGBM_VIX Log Value,<optuna.study.study.Study object at 0x000002AA...,LGBMRegressor(colsample_bytree=0.6174407481512...,0.074313,0.004452


In [434]:
# Calculate the median of the R2 score for each model name
model_result.groupby('Name')['R2'].median()

Name
LightGBM_All              0.051275
LightGBM_Baseline        -0.176613
LightGBM_SOX RV Lag      -0.033237
LightGBM_SOX Spread      -0.022859
LightGBM_VIX Log Value   -0.236434
Name: R2, dtype: float64

In [430]:
# Calculate the median of the RMSE score for each model name
model_result.groupby('Name')['RMSE'].median()

Name
LightGBM_All              0.072541
LightGBM_Baseline         0.080788
LightGBM_SOX RV Lag       0.075700
LightGBM_SOX Spread       0.075324
LightGBM_VIX Log Value    0.082817
Name: RMSE, dtype: float64